<a href="https://colab.research.google.com/github/silveira-alysson/supply-network-analysis/blob/main/SupplyChain_NetworkAnalysis_30Oct25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Alysson Silveira**
**10-Aug-2025**
## My Procedure to Clean Up the Revere–FactSet Supply Chain Dataset

## Data Cleaning and Construction Steps

1. **Add GVKEY to the dyad dataset**  
   This step is performed early because multiple `source_company_id` and `target_company_id` values can map to a single CUSIP (e.g., subsidiaries).  
   Adding GVKEY at this stage allows aggregation across all company IDs under the same GVKEY, avoiding duplication.

2. **Extract year variables**  
   Extracted `start_year` and `end_year` from the start and end date fields.

3. **Handle missing end dates**  
   Missing end dates are assumed to represent ongoing relationships.  
   Missing `end_year` values were filled with **2022**, which is outside the data range (2005–2021).

4. **Create a pseudo-panel**  
   Generated one observation (row) for **each year between `start_year` and `end_year`** for every relationship.

5. **Filter relationship types**  
   Retained only:
   - `REL_TYPE = "CUSTOMER"`
   - `REL_TYPE = "SUPPLIER"`

6. **Invert relationships to capture all dyads**  
   The dataset lists relationships from the focal firm’s perspective, but firms may also appear as customers or suppliers of other focal firms.  
   To ensure all relationships are captured:
   - Created a copy of the dataset
   - Inverted key fields:
     - `source_name`, `source_id`, `source_cusip` → `target_name`, `target_id`, `target_cusip`
     - `REL_TYPE = "CUSTOMER"` → `"SUPPLIER"`
     - `REL_TYPE = "SUPPLIER"` → `"CUSTOMER"`

7. **Append datasets**  
   Appended the original and inverted datasets.

8. **Drop duplicates**

9. **Retain only customer relationships**  
   Kept only `REL_TYPE = "CUSTOMER"`.

   **Example**

   **Original Table**

   | Source | Target | Relationship |
   |-------:|-------:|-------------|
   | X      | Y      | Customer    |
   | X      | Z      | Supplier    |

   **Inverted Table**

   | Source | Target | Relationship |
   |-------:|-------:|-------------|
   | Y      | X      | Supplier    |
   | Z      | X      | Customer    |

   **Appended Table**

   | Source | Target | Relationship |
   |-------:|-------:|-------------|
   | X      | Y      | Customer    |
   | X      | Z      | Supplier    |
   | Y      | X      | Supplier    |
   | Z      | X      | Customer    |

10. **Build directed graph**  
    Constructed a directed graph where edges always go from:
    - **Source = supplier**
    - **Target = customer**

    - **203,026 nodes**
    - **749,168 edges**

11. **Build one graph per year**

12. **Compute node-level network measures (per year)**  
    - Reciprocity  
    - In-degree  
    - Out-degree  
    - In-degree centrality  
    - Out-degree centrality  
    - Degree centrality  
    - Eigenvector centrality  
    - Clustering coefficient  
    - Core number  

13. **Add firm identifier**  
    Added the node’s (source company) **CUSIP** to the output table.

14. **Export final dataset**  
    Exported the resulting dataset as a `.dta` file.



In [ ]:
#import packages
import pandas as pd
import networkx as nx

In [ ]:
!wget https://www.dropbox.com/scl/fi/v25zqpcj2g9sfwr928fcs/seglink_withSIC_30Oct25.dta?rlkey=19q6hymbsf5osmxi8pdy1rkpj&dl=0

In [ ]:
##rename file to get rid of the weird extension name

In [ ]:
df = pd.read_stata("seglink_withSIC_30Oct25.dta?rlkey=19q6hymbsf5osmxi8pdy1rkpj")

In [ ]:
df

In [ ]:
#Create year column based on srcdate
df["year"] = df['srcdate'].dt.year

In [ ]:
#Inspect df
df.columns

In [ ]:
#create directed graph
G = nx.from_pandas_edgelist(
    df,
    source="gvkey",
    target="cgvkey",
    edge_attr=["salecs", "year"],
    create_using=nx.DiGraph(),
)

In [ ]:
#check number of nodes and edges
print(f"number of nodes: {len(G.nodes)}")
print(f"number of edges: {len(G.edges)}")

In [ ]:
yearly_subgraphs = {}
for year in range(2003, 2020):

    # Create the directed graph from your dataframe (replace 'new_panel_df_Customer' with your actual dataframe)
    yearly_subgraphs[year] = nx.from_pandas_edgelist(
                              df[df['year']==year],
                              source="gvkey",
                              target="cgvkey",
                              edge_attr=["SOURCE_it", "SOURCE_it_hdwr", "SOURCE_ict", "SOURCE_ict_hdwr", "year"],
                              create_using=nx.DiGraph(),
                          )

    #Create year attribute
    nx.set_node_attributes(yearly_subgraphs[year], year, name=f'year')

    # Reciprocity
    reciprocity_values = {node: nx.reciprocity(yearly_subgraphs[year], node) for node in yearly_subgraphs[year].nodes()}
    nx.set_node_attributes(yearly_subgraphs[year], reciprocity_values, name=f'reciprocity')

    # Compute in-degree and out-degree as dictionaries {node: degree}
    in_degree_values = dict(yearly_subgraphs[year].in_degree())
    nx.set_node_attributes(yearly_subgraphs[year], in_degree_values, name='in_degree')

    # Set only the degree value (not the full list of tuples)
    out_degree_values = dict(yearly_subgraphs[year].out_degree())
    nx.set_node_attributes(yearly_subgraphs[year], out_degree_values, name='out_degree')


    # Compute in-degree and out-degree if SOURCE_IT == 1
    edges_it = [(u, v) for u, v, d in yearly_subgraphs[year].edges(data=True) if d.get('SOURCE_it') == 1]
    subgraph_it = yearly_subgraphs[year].edge_subgraph(edges_it).copy()
    in_degree_values_it = dict(subgraph_it.in_degree(weight=None))
    nx.set_node_attributes(yearly_subgraphs[year], in_degree_values_it, name=f'in_degree_it')
    out_degree_values_it = dict(subgraph_it.out_degree(weight=None))
    nx.set_node_attributes(yearly_subgraphs[year], out_degree_values_it, name=f'out_degree_it')
    print(f"In and Out Degree Source IT for {year} calculated.")

    # Compute in-degree and out-degree if SOURCE_IT_HDWR == 1
    edges_it_hdwr = [(u, v) for u, v, d in yearly_subgraphs[year].edges(data=True) if d.get('SOURCE_it_hdwr') == 1]
    subgraph_it_hdwr = yearly_subgraphs[year].edge_subgraph(edges_it_hdwr).copy()
    in_degree_values_it_hdwr = dict(subgraph_it_hdwr.in_degree(weight=None))
    nx.set_node_attributes(yearly_subgraphs[year], in_degree_values_it_hdwr, name=f'in_degree_it_hdwr')

    out_degree_values_it_hdwr = dict(subgraph_it_hdwr.out_degree(weight=None))
    nx.set_node_attributes(yearly_subgraphs[year], out_degree_values_it_hdwr, name=f'out_degree_it_hdwr')
    print(f"In and Out Degree Source IT HDWR for {year} calculated.")

    # Compute in-degree and out-degree if SOURCE_ICT == 1
    edges_ict = [(u, v) for u, v, d in yearly_subgraphs[year].edges(data=True) if d.get('SOURCE_ict') == 1]
    subgraph_ict = yearly_subgraphs[year].edge_subgraph(edges_ict).copy()
    in_degree_values_ict = dict(subgraph_ict.in_degree(weight=None))
    nx.set_node_attributes(yearly_subgraphs[year], in_degree_values_ict, name=f'in_degree_ict')

    out_degree_values_ict = dict(subgraph_ict.out_degree(weight=None))
    nx.set_node_attributes(yearly_subgraphs[year], out_degree_values_ict, name=f'out_degree_ict')
    print(f"In and Out Degree Source ICT for {year} calculated.")

    # # Compute in-degree and out-degree if TARGET_ICT_HDWR == 1
    edges_ict_hdwr = [(u, v) for u, v, d in yearly_subgraphs[year].edges(data=True) if d.get('SOURCE_ict_hdwr') == 1]
    subgraph_ict_hdwr = yearly_subgraphs[year].edge_subgraph(edges_ict_hdwr).copy()
    in_degree_values_ict_hdwr = dict(subgraph_ict_hdwr.in_degree(weight=None))
    nx.set_node_attributes(yearly_subgraphs[year], in_degree_values_ict_hdwr, name=f'in_degree_ict_hdwr')

    out_degree_values_ict_hdwr = dict(subgraph_ict_hdwr.out_degree(weight=None))
    nx.set_node_attributes(yearly_subgraphs[year], out_degree_values_ict_hdwr, name=f'out_degree_ict_hdwr')
    print(f"In and Out Degree Source IT HDWR for {year} calculated.")


    # In-degree and Out-degree Centrality
    in_degree_centrality_values = nx.in_degree_centrality(yearly_subgraphs[year])
    nx.set_node_attributes(yearly_subgraphs[year], in_degree_centrality_values, name=f'in_degree_centrality')

    out_degree_centrality_values = nx.out_degree_centrality(yearly_subgraphs[year])
    nx.set_node_attributes(yearly_subgraphs[year], out_degree_centrality_values, name=f'out_degree_centrality')

    # Betweenness Centrality
    betweenness_centrality_values = nx.betweenness_centrality(yearly_subgraphs[year], seed=17)
    nx.set_node_attributes(yearly_subgraphs[year], betweenness_centrality_values, name=f'betweenness_centrality')

    # Closeness Centrality
    closeness_centrality_values = nx.closeness_centrality(yearly_subgraphs[year])
    nx.set_node_attributes(yearly_subgraphs[year], closeness_centrality_values, name=f'closeness_centrality')

    # Eigenvector Centrality
    try:
        eigenvector_centrality_values = nx.eigenvector_centrality(yearly_subgraphs[year], max_iter=1000)
    except nx.PowerIterationFailedConvergence:
        # Handle case where eigenvector centrality fails to converge
        eigenvector_centrality_values = {node: float('nan') for node in yearly_subgraphs[year].nodes()}
    nx.set_node_attributes(yearly_subgraphs[year], eigenvector_centrality_values, name=f'eigenvector_centrality')

    # PageRank
    pagerank_values = nx.pagerank(yearly_subgraphs[year])
    nx.set_node_attributes(yearly_subgraphs[year], pagerank_values, name=f'pagerank')

    # Clustering Coefficient
    clustering_coefficient_values = nx.clustering(yearly_subgraphs[year])
    nx.set_node_attributes(yearly_subgraphs[year], clustering_coefficient_values, name=f'clustering_coefficient')


    # Assortativity (for directed degree distributions)
    try:
        assortativity_coefficient = nx.degree_pearson_correlation_coefficient(yearly_subgraphs[year])
    except nx.NetworkXError:
        # Handle the case where assortativity cannot be calculated
        assortativity_coefficient = float('nan')  # Assign NaN if calculation fails

    # Set it for all nodes to keep consistent structure
    nx.set_node_attributes(yearly_subgraphs[year], {n: assortativity_coefficient for n in yearly_subgraphs[year].nodes()}, name=f'assortativity_coefficient')

    # # Information_centrality (only available for undirected graphs)
    # information_centrality_values = nx.information_centrality(yearly_subgraphs[year])
    # nx.set_node_attributes(yearly_subgraphs[year], information_centrality_values, name=f'information_centrality')

    # degree_centrality
    degree_centrality_values = nx.degree_centrality(yearly_subgraphs[year])
    nx.set_node_attributes(yearly_subgraphs[year], degree_centrality_values, name=f'degree_centrality')

    print(f"Metrics for year {year} calculated and added to graph.")






# Now gather all the nodes and their attributes into a long format

# Define the column names
columns = ['node', 'year', 'reciprocity', 'in_degree', 'out_degree',
           'in_degree_it', 'out_degree_it',
           'in_degree_it_hdwr', 'out_degree_it_hdwr',
           'in_degree_centrality', 'out_degree_centrality',
           'betweenness_centrality', 'closeness_centrality', 'eigenvector_centrality',
           'pagerank', 'clustering_coefficient', 'assortativity_coefficient', 'degree_centrality']




In [ ]:
# Create an empty DataFrame
dfNet = pd.DataFrame(columns=columns)

for year in range(2003, 2020):
  # Step 1: Get nodes with their attributes (metadata)
  node_data = yearly_subgraphs[year].nodes(data=True)

  # Step 2: Convert nodes with metadata to a list of dictionaries
  nodes_list = [{**{'node': n}, **d} for n, d in node_data]

  # Step 3: Convert the list of dictionaries to a Pandas DataFrame
  dfNet = pd.concat([dfNet, pd.DataFrame(nodes_list)], ignore_index=True)

  #Enforce fields as numeric
  dfNet.year = dfNet.year.astype(int)
  dfNet.in_degree = dfNet.in_degree.astype(int)
  dfNet.out_degree = dfNet.out_degree.astype(int)
  dfNet.in_degree_it = dfNet.in_degree_it.astype('Int64')
  dfNet.out_degree_it = dfNet.out_degree_it.astype('Int64')
  dfNet.in_degree_it_hdwr = dfNet.in_degree_it_hdwr.astype('Int64')
  dfNet.out_degree_it_hdwr = dfNet.out_degree_it_hdwr.astype('Int64')

  #replace NAs with zero
  dfNet.fillna(0, inplace=True)

In [ ]:
#Add Cusip to the dataframe

cusip = df[['gvkey',	'scusip']].drop_duplicates().set_index('gvkey').to_dict()['scusip']
dfNet['cusip'] = dfNet['node'].map(cusip)


In [ ]:
dfNet['in_degree'] = pd.to_numeric(dfNet['in_degree'], errors='coerce')
dfNet['out_degree'] = pd.to_numeric(dfNet['out_degree'], errors='coerce')

In [ ]:
dfNet.year = dfNet.year.astype(int)
dfNet['node'] = dfNet['node'].astype(str)
dfNet.to_stata('/content/drive/MyDrive/FactsetSupplyChainNetwork/networkMeasuresWRDS_SIC_2003_2019_30Oct25v1aos.dta')

In [ ]:
###End of Data Modelling.
###Begin Data Exploration and Small Supply Network Visualization

In [ ]:
##Examples APPLE

In [ ]:
df[df['gvkey']=='001690']

In [ ]:
df[df['cgvkey']=='001690']

In [ ]:
dfNet[dfNet['node']=='1690']

In [ ]:
node_reciprocity = {node: nx.reciprocity(G, node) for node in G.nodes()}

In [ ]:
import matplotlib.pyplot as plt
plt.hist(reciprocity_values.values(), bins=20)
plt.xlabel('Reciprocity Value')
plt.ylabel('Frequency')
plt.title('Distribution of Reciprocity Values')
plt.show()

In [ ]:
df_choice = df[(df['gvkey']=='287882') | (df['cgvkey']=='287882') | (df['gvkey']=='164046') | (df['cgvkey']=='164046') | (df['gvkey']=='278151') | (df['cgvkey']=='278151') | (df['gvkey']=='318221') | (df['cgvkey']=='318221')]

In [ ]:
df_choice

In [ ]:
pep_yearly_subgraphs = {}
for year in range(2017, 2020):
    # Create the directed graph from your dataframe (replace 'new_panel_df_Customer' with your actual dataframe)
    pep_yearly_subgraphs[year] = nx.from_pandas_edgelist(
                              df_choice[df_choice['year']==year],
                              source="gvkey",
                              target="cgvkey",
                              edge_attr=["year"],
                              create_using=nx.DiGraph(),
                          )

In [ ]:
nx.draw(pep_yearly_subgraphs[2017], with_labels=True, node_color='lightblue', node_size=500)

In [ ]:
nx.draw(pep_yearly_subgraphs[2018], with_labels=True, node_color='lightblue', node_size=500)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
nx.draw(pep_yearly_subgraphs[2019], with_labels=True, node_color='lightblue', node_size=500)

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = pep_yearly_subgraphs[2018]

# group nodes by column
very_top = ["009835"]
top_nodes = ["164046", "278151"]
middle_nodes = ["287882", "013971"]
bottom_nodes = ["318221"]
options = {
    "font_size": 12,
    "node_size": 2500,
    "node_color": "white",
    "edgecolors": "black",
    "linewidths": 1,
    "width": 1,
}

# set the position according to column (x-coord)
pos = {n: (i, 0) for i, n in enumerate(very_top)}
pos.update({n: (i, -1) for i, n in enumerate(top_nodes)})
pos.update({n: (i, -2) for i, n in enumerate(middle_nodes)})
pos.update({n: (i, -3) for i, n in enumerate(bottom_nodes)})

nx.draw_networkx(G, pos, **options)

#highlight one node by changing the color of the line
nx.draw_networkx_nodes(G, pos, nodelist=["287882"], node_color="red", node_size=2500)

# Set margins for the axes so that nodes aren't clipped
ax = plt.gca()
ax.margins(0.1)
plt.axis("off")
plt.show()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.drawing.layout import spring_layout

G = pep_yearly_subgraphs[2019]

# group nodes by column
very_top = ["009835"]
top_nodes = ["164046", "278151"]
middle_nodes = ["287882", "013971"]
bottom_nodes = ["318221"]
options = {
    "font_size": 12,
    "node_size": 2500,
    "node_color": "white",
    "edgecolors": "black",
    "linewidths": 1,
    "width": 1,
}

# set the position according to column (x-coord)

pos = {n: (i, 0) for i, n in enumerate(very_top)}
pos.update({n: (i, -1) for i, n in enumerate(top_nodes)})
pos.update({n: (i, -2) for i, n in enumerate(middle_nodes)})
pos.update({n: (i, -3) for i, n in enumerate(bottom_nodes)})

nx.draw_networkx(G, pos, **options)

#highlight one node by changing the color of the line
nx.draw_networkx_nodes(G, pos, nodelist=["287882"], node_color="red", node_size=2500)

# Set margins for the axes so that nodes aren't clipped
ax = plt.gca()
ax.margins(0.1)
plt.axis("off")
plt.show()